In [ ]:

def getSafeDurationFromVideos(video_path):

    # fore resizing a window for video (optional depending on screen size)
    cv2.namedWindow('Video', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Video', height=760, width=1366)

    # setting font properties for displaying frame number with video
    font = cv2.FONT_HERSHEY_COMPLEX  # fontFace
    fontScale = 1  # fontScale
    fontColor = (255,255,255) # fontColor
    lineType = 2 # lineType

    # capture a video from given path
    scanned_video = cv2.VideoCapture(video_path)
    frame_count = -1

    ss = []  # safe duration start list
    se = [] # safe duration end list
    safe_duration_list = [] # final list containing ss and se

    if (scanned_video.isOpened()== False): # check if there is error in opening a video
        raise Exception("check the videopath or error opening video")

    while(scanned_video.isOpened()):

        ret, frame = scanned_video.read() 
        key = cv2.waitKey(100)
        
        if(ret == True):
            
            frame_count = frame_count + 1
            
            cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
            cv2.imshow('Video',frame)

            if key == ord('p'): # pause a video

                while True:
                    
                    key2 = cv2.waitKey(25)

                    cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
                    cv2.imshow('Video', frame)
                    
                    if key2 == ord('p'): # resume video after pausing
                        break
            
            if key == ord('s'): # save safe duration start

                ss.append(frame_count)
                safe_duration_list.append(frame_count)
                
            if key == ord('e'): # save safe duration end

                se.append(frame_count)
                safe_duration_list.append(frame_count)
            
            if key == ord('q'): #quit a video
                break

        else:
            break


    scanned_video.release()
    cv2.destroyAllWindows()


    return safe_duration_list

In [13]:
import csv
import cv2
import glob
import natsort

def save_labels_csv(videos_list, csv_file):
    
    csv_writer = csv.writer(open(csv_file,"w"))    
    
    for video_path in videos_list:
        
        ready_flag = str.lower(input('type y if you are ready to annotate this video and n to quit the program'))
        if(ready_flag == 'y'):
            
            surity_flag = 0
            while(surity_flag == 0):
                safe_duration_list = getSafeDurationFromVideos(video_path)
                inp = int(input('Enter 0 to repeat the annotations process and 1 to proceed to next video'))
                if(inp == 1):
                    break
            csv_writer.writerow(safe_duration_list)
            print(safe_duration_list)
            
        else:
            
            print('Take a break! You must be tired')
            break


video_folder = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos_all/'
video_list = glob.glob(video_folder + '/video*.MOV')
video_list = natsort.natsorted(video_list)

# video_list = ['/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos_train/video1.MOV']

csv_file = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels-frame-wise.csv'

save_labels_csv(video_list, csv_file)

[26, 52, 151, 197]


In [2]:
# import cv2


# video_path = "/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos_train/video1.MOV"

# # fore resizing a window for video (optional depending on screen size)
# cv2.namedWindow('Video', cv2.WINDOW_NORMAL)
# cv2.resizeWindow('Video', height=760, width=1366)

# # setting font properties for displaying frame number with video
# font = cv2.FONT_HERSHEY_COMPLEX  # fontFace
# fontScale = 1  # fontScale
# fontColor = (255,255,255) # fontColor
# lineType = 2 # lineType

# # capture a video from given path
# scanned_video = cv2.VideoCapture(video_path)
# frame_count = -1

# ss = []  # safe duration start list
# se = [] # safe duration end list
# safe_duration_list = [] # final list containing ss and se

# if (scanned_video.isOpened()== False): # check if there is error in opening a video
#     raise Exception("check the videopath or error opening video")

# while(scanned_video.isOpened()):

#     ret, frame = scanned_video.read() 
#     key = cv2.waitKey(100)
    
#     if(ret == True):
        
#         frame_count = frame_count + 1
        
#         cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
#         # cv2.imshow('Video', frame)
#         cv2.setWindowTitle('Video', 'Video1')
#         cv2.imshow('Video',frame)

#         if key == ord('p'): # pause a video

#             while True:
                
#                 key2 = cv2.waitKey(25)

#                 cv2.putText(frame, "Frame Number:" + str(frame_count), org=(20,50), fontFace=font, fontScale=fontScale, color=fontColor, lineType=lineType, thickness=3)
#                 cv2.setWindowTitle('Video', 'Video1')
#                 cv2.imshow('Video', frame)
                
#                 if key2 == ord('p'): # resume video after pausing
#                     break
        
#         if key == ord('s'): # save safe duration start

#             ss.append(frame_count)
#             safe_duration_list.append(frame_count)
            
#         if key == ord('e'): # save safe duration end

#             se.append(frame_count)
#             safe_duration_list.append(frame_count)
        
#         if key == ord('q'): #quit a video
#             break

#     else:
#         break


# scanned_video.release()
# cv2.destroyAllWindows()

# print(ss)
# print(se)
# print(safe_duration_list)



104
